## 모델 불러오기

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install transformers;
!pip install sentencepiece;

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 107.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 30.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 125.2 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 47.4 MB/s eta 0:00:00


In [3]:
from transformers import (
    PreTrainedTokenizerFast as BaseGPT2Tokenizer,
    EncoderDecoderModel,
    DataCollatorForSeq2Seq,
)
import pickle
from drive.MyDrive.WARNING_PRIVATE_FOLDER.OSSP2AntiGPT9.ai.tokenization_kobert import KoBertTokenizer
#from drive.MyDrive.WARNING_PRIVATE_FOLDER.OSSP2AntiGPT9.ai.lib import tokenization_kobert
import json
src_tokenizer = KoBertTokenizer.from_pretrained('monologg/kobert')

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'KoBertTokenizer'.


In [4]:
class GPT2Tokenizer(BaseGPT2Tokenizer):
    def build_inputs_with_special_tokens(self, token_ids, _):
        return token_ids + [self.eos_token_id]
trg_tokenizer = GPT2Tokenizer.from_pretrained("skt/kogpt2-base-v2",
  bos_token='</s>', eos_token='</s>', unk_token='<unk>',
  pad_token='<pad>', mask_token='<mask>')

In [5]:
%cd /content/drive/MyDrive/WARNING_PRIVATE_FOLDER/OSSP2AntiGPT9/ai
# with open('/content/drive/MyDrive/WARNING_PRIVATE_FOLDER/OSSP2AntiGPT9/ai/no_commit/save_src_tk.pickle','rb') as f:
#     src_tokenizer = pickle.load(f)
# there's an error in this code when activating in local

/content/drive/MyDrive/WARNING_PRIVATE_FOLDER/OSSP2AntiGPT9/ai


In [6]:
with open("/content/drive/MyDrive/WARNING_PRIVATE_FOLDER/OSSP2AntiGPT9/ai/standard_to_dialect/no_commit/added_tokens_trg.json", 'rb') as f:
    dw_dict = json.load(f)
dict_swap = {v:k for k,v in dw_dict.items()}
ma=54000
mi=51200
for i in range(51200,54001):
    trg_tokenizer.add_tokens(dict_swap[i])

In [7]:
model = EncoderDecoderModel.from_pretrained('leadawon/ossp-reverse-v0_3')
model.eval()
model.config.decoder_start_token_id = trg_tokenizer.bos_token_id

## 테스트데이터 불러오기

In [8]:
def read_input(path):
    with open(path, 'rb') as f:
        test_dict = json.load(f)
    test_standard = []
    test_dialect = []
    for pairs in test_dict['utterance']:
        test_standard.append(pairs['standard_form'])
        test_dialect.append(pairs['dialect_form'])

    return test_standard ,test_dialect

In [9]:
test_s ,test_d= read_input("/content/drive/MyDrive/WARNING_PRIVATE_FOLDER/OSSP2AntiGPT9/ai/no_commit/test_data_big.json")

In [10]:
def translate(text):
    embeddings = src_tokenizer(text, return_attention_mask=False, return_token_type_ids=False, return_tensors='pt')
    embeddings = {k: v for k, v in embeddings.items()}
    output = model.generate(**embeddings)[0, 1:-1]
    tmp = trg_tokenizer.decode(output.cpu())
    return tmp

In [11]:
print("size of full : ",len(test_s))

size of full :  12451


In [12]:
for i in range(len(test_d)):
    sr_text = test_s[i] # standard
    tg_text = translate(sr_text) # translate
    print("source_text : ",sr_text)
    print("result :      ",tg_text)
    print("ground_truth :",test_d[i])
    print()
    print()
    if i%50==0:
        print("iteration num : ",i)
        break

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1255: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1346: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


source_text :  언니 만나서 반가워 아 오늘 제주 방언 에이 아이 데이터
result :       언니 만낭 반가 워 아 오늘 제주 방언 에이 아이 데이터    
ground_truth : 언니 만낭 반가워 아 오늘 제주 방언 에이 아이 데이터


iteration num :  0


## scoring

In [13]:
!pip install "sacrebleu[ko]"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 582.5/582.5 kB 50.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.2/33.2 MB 18.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for mecab-ko-dic: filename=mecab_ko_dic-1.0.0-py3-none-any.whl size=33424394 sha256=7a57d92e124b208f4ea9ebb955ec7af8d0a88c7049e784ab85c88eec11d52572
  Stored in directory: /root/.cache/pip/wheels/0d/46/b8/996432c9998690a5a66fb82f6499222e48a23ff5b6c7d80322
Successfully built mecab-ko-dic


In [14]:
from sacrebleu.metrics import BLEU

In [15]:
def score_B(sr_text, tg_text):

    candidate=[tg_text]
    reference=[[sr_text]]

    bleu = BLEU()

    bleuscore = bleu.corpus_score(candidate,reference).score
    return bleuscore
    #bleu_save.append(bleuscore)

In [16]:
sumation = 0
NUM_OF_TEST = 500
for i in range(len(test_s)):
    sr_text = test_s[i] # standard
    tg_text = translate(sr_text) # translate
    print("source_text : ",sr_text)
    print("result      : ",tg_text)
    print("ground_truth: ",test_d[i])
    scr = score_B(tg_text.strip(),test_d[i].strip())
    sumation += scr
    print("sum_score : {}".format(sumation/(i+1)))
    print("bleu_socre  :  {:.2f}".format(scr))
    print()
    print()
    if i%50==0:
        print("iteration num : ",i)
    if i==NUM_OF_TEST:
        break
print("at_last_sum_score : {}".format(sumation/NUM_OF_TEST))

source_text :  언니 만나서 반가워 아 오늘 제주 방언 에이 아이 데이터
result      :  언니 만낭 반가 워 아 오늘 제주 방언 에이 아이 데이터    
ground_truth:  언니 만낭 반가워 아 오늘 제주 방언 에이 아이 데이터
sum_score : 63.981667416455416
bleu_socre  :  63.98


iteration num :  0
source_text :  어 명절 설 명절 추석 명절 요로케 나누어서 해볼 거예요
result      :  어 맹질 설 맹질 추석 맹질 요로케 갈랑 해볼 거
ground_truth:  어 명절 설 명절 추석 명절 요로케 나누어서 해볼 거예
sum_score : 35.12816136372508
bleu_socre  :  6.27


source_text :  자 그러면 이제부터 얘기해 봐요
result      :  자 그러면 이제부터 얘기 해 봅서        
ground_truth:  자 그믄 이제부터 얘기해 보게예
sum_score : 26.885640138818488
bleu_socre  :  10.40


source_text :  어 그러면 언니네 설 명절 때 음식 어떻게 해
result      :  어 게믄 언니네 설 명절 때 음식 어떵 해     
ground_truth:  어 그믄 언니네 설 명절 때 음식 어떵 해
sum_score : 38.92982644787236
bleu_socre  :  75.06


source_text :  음식을 막 준비했던 기억이 나
result      :  음식을 막 준비했던 기억이 나맨          
ground_truth:  음식을 막 준비했던 기억이 나게
sum_score : 44.51866725782632
bleu_socre  :  66.87


source_text :  밀가룬지 쌀가룬지를 모르겠어  근데 그거를 푹
result      :  밀가루사 쌀가룬지를 모르크라 근데 그거를 푹    
ground_tru